# Residual Networks 

Implementation of Residual networks (layer blocks with skip connections).  

In [1]:
import tensorflow as tf

In [2]:
import numpy as np 

from matplotlib.pyplot import imshow
%matplotlib inline

import tensorflow.keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Earlier layers in deep NN architectures have a harder time being trained because of the vanishing gradient problem.  Skip connections help with this by allowing the gradients another way to pass backwards through the network.  The skip connections also easily allow for the network to learn an identify function, so stacking multiple skip connection blocks is not harmful for the model.   

## Identity block 

In [4]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add 

In [13]:
def identity_block(X, f, filters, stage, block): 
    """  
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis 
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # retrieve filters 
    F1, F2, F3 = filters 
    
    # save the input value.  this will be skipped forward 
    X_shortcut = X  
    
    # first component of main path 
    X = Conv2D(filters=F2, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', 
              kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X) 
    X = Activation('relu')(X) 
    
    
    # second component of main path 
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', 
              kernel_initializer=tf.keras.initializers.glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X) 
    X = Activation('relu')(X) 
       
        
    # third component of main path 
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', 
               kernel_initializer = tf.keras.initializers.glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    
    # add shortcut back in 
    X = Add()([X, X_shortcut])  
    X = Activation('relu')(X)  
    
    return X 
    
    

In [16]:
tf.compat.v1.reset_default_graph() 

with tf.compat.v1.Session() as test: 
    np.random.seed(1) 
    A_prev = tf.compat.v1.placeholder('float', [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    
    A = identity_block(A_prev, f=2, filters=[2, 4, 6], stage=1, block='a')
    
    test.run(tf.compat.v1.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    
    print(str(out[0][1][1][0]))

[0.        0.        1.2737559 1.9671017 0.        1.236164 ]


In [21]:
np.array(X).shape

(3, 4, 4, 6)

# Reference 
+ https://github.com/calmisential/TensorFlow2.0_ResNet  
+ https://tensorflow.google.cn/tutorials/quickstart/advanced 